DATA ENGINEER - PYTHON PYSPARK

This test consits of fifteen problems. You are required to write your code in cell below each problem and output the result in cell next to it 

Total Time Allowed: 3 hours

Style1: 10 markes each shall be awarded for correct solution coded in object oriented paradign in Python3 <br>
Style2: 15 marks each shall be awarded for correct solution coded in functional programming paradigm (lamda, map, reduce, filter etc) in Python3 <br>
Style3: 20 marks each shall be awarded for correct solution coded in functional programming paradigm (dataframes, map, reduce, filter etc) in PySpark3

Rename and Save the notebook with your FirstName_LastName (eg. Sahil_Gupta.ipynb)

In [1]:
#Vikas Veerabhadra Chitturi
#vikasch.1994@gmail.com
#+91-9963844373

********************************************* Test starts here **************************************************

INSTRUCTIONS:
    1. You are required to download and import five CSV files, one json file and one xml file
    2. You would need to understand business involved behind CRM database tables. This is important
    3. Code must be in Python3/PySpark3
    4. Either your code should output something or leave the comment "#solution code here" as it is. We shall use 'Run All' in notebook and it shouldn't result error
    5. Test the entire notebook before uploading to Google Form provided
    6. You can use any Python3 library (two are imported already) or PySpark3 library. There is no restriction
    7. Output fieldname to be displayed are marked as single quaotes '' in problem statement. You should use same field alias names whereever required
    8. Notation for dataframe and/or array must be local to a problem's solution. Eg. Dataframe "test" for problem 8 should be df_prb8_test

In [2]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from collections import defaultdict
import json

In [3]:
#Please make sure this notebook is run on spark installed cluster/environment - Data processing is done in pyspark
spark = SparkSession.builder\
          .master("local[4]")\
          .appName("Assignment")\
          .getOrCreate()
print (spark)

In [4]:
#import CSVs here
df_sales_teams = spark.read.format('csv')\
                      .option("header","true")\
                      .load("data/sales_teams.csv")
df_accounts = spark.read.format('csv')\
                      .option("header","true")\
                      .load("data/accounts.csv")
df_clicks = spark.read.format('csv')\
                      .option("header","true")\
                      .load("data/clicks.csv")
df_products = spark.read.format('csv')\
                      .option("header","true")\
                      .load("data/products.csv")
df_sales_pipeline = spark.read.format('csv')\
                      .option("header","true")\
                      .load("data/sales_pipeline.csv")

print ("Sales Teams...")
print (df_sales_teams.take(5))

print ("Accounts....")
print (df_accounts.take(5))

print ("Clicks....")
print (df_clicks.take(5))

print ("Products....")
print (df_products.take(5))

print ("Sales Pipeline....")
print (df_sales_pipeline.take(5))

#import JSONs here

Sales Teams...
[Row(sales_agent='Donn Cantrell', manager='Rocco Neubert', regional_office='Central'), Row(sales_agent='James Ascencio', manager='Summer Sewald', regional_office='West'), Row(sales_agent='Vicki Laflamme', manager='Celia Rouche', regional_office='West'), Row(sales_agent='Niesha Huffines', manager='Melvin Marxen', regional_office='East'), Row(sales_agent='Kami Bicknell', manager='Summer Sewald', regional_office='West')]
Accounts....
[Row(account='Sunnamplex', revenue='4592.96', employees='13938.0'), Row(account='Silis', revenue='5339.57', employees='18053.0'), Row(account='Groovestreet', revenue='2728.86', employees='6486.0'), Row(account='Donware', revenue='2009.52', employees='3409.0'), Row(account='Wonka Industries', revenue='4962.27', employees='4687.0')]
Clicks....
[Row(created_on='2016-11-14', source='Referral', industry='IT'), Row(created_on='2016-11-14', source='Social', industry='IT'), Row(created_on='2016-11-14', source='Paid', industry='SaaS'), Row(created_on='2

Refer & Use five CSVs to answer problem 1-10 below

PROBLEM 1: Display 'Manager' and 'Grand Total Sales', for sales done by the sales agents reporting these managers

In [5]:
#solution code here
#joining sales_teams and sales_pipeline tables
df_st_sp_prob1 = df_sales_teams.select("sales_agent","manager").alias("a")\
                         .join(df_sales_pipeline.select("sales_agent","deal_stage","close_value").alias("b"),
                               on=col("a.sales_agent")==col("b.sales_agent"),how='left')\
                         .select([col("a.sales_agent"),col("a.manager"),col("b.deal_stage"),col("b.close_value")])
df_st_sp_prob1.show(10) 
print (df_st_sp_prob1.count())
#putting a filter on deal_stage=='Won' because it is considered as successful sale completed and close_value is the
#final value for which product is sold after negotiation
df_st_sp_prob1 = df_st_sp_prob1.filter(col("deal_stage")=="Won")
df_st_sp_prob1 = df_st_sp_prob1.withColumn("close_value",df_st_sp_prob1["close_value"].cast(DoubleType()))
result_prob1 = df_st_sp_prob1.groupBy("manager").agg(sum("close_value").alias("grand_total_sales"))
result_prob1.show()

+-------------+-------------+-----------+-----------+
|  sales_agent|      manager| deal_stage|close_value|
+-------------+-------------+-----------+-----------+
|Donn Cantrell|Rocco Neubert|        Won|      444.0|
|Donn Cantrell|Rocco Neubert|In_Progress|       null|
|Donn Cantrell|Rocco Neubert|       Lost|        0.0|
|Donn Cantrell|Rocco Neubert|        Won|     7695.0|
|Donn Cantrell|Rocco Neubert|        Won|     5531.0|
|Donn Cantrell|Rocco Neubert|In_Progress|       null|
|Donn Cantrell|Rocco Neubert|        Won|     7381.0|
|Donn Cantrell|Rocco Neubert|        Won|      565.0|
|Donn Cantrell|Rocco Neubert|In_Progress|       null|
|Donn Cantrell|Rocco Neubert|        Won|     7905.0|
+-------------+-------------+-----------+-----------+
only showing top 10 rows

14277
+----------------+-----------------+
|         manager|grand_total_sales|
+----------------+-----------------+
|    Celia Rouche|        2518466.0|
|   Rocco Neubert|        3346813.0|
|   Melvin Marxen|        4

PROBLEM 2: Display 'Sales Agents' and 'Sales' for those sales where product sold at profit

In [6]:
#solution code here
#joining sales_pipeline and products tables to get the mapping of actual and final prices
df_sp_prod_prob2 = df_sales_pipeline.filter(col("deal_stage")=="Won")\
                         .select("opportunity_id","sales_agent","deal_stage","product","close_value").alias("a")\
                         .join(df_products.select("product","sales_price").alias("b"),
                               on=col("a.product")==col("b.product"),how='left')\
                         .select([col("a.opportunity_id"),col("a.sales_agent"),col("a.deal_stage"),col("a.product"),
                                  col("a.close_value").alias("final_price"),
                                  col("b.sales_price").alias("actual_price")])
df_sp_prod_prob2.show(20)
print (df_sp_prod_prob2.count())

#profit means final price (close_value) is greater than actual price (sales_price)
result_prob2 = df_sp_prod_prob2.where(col("final_price") > col("actual_price"))
result_prob2 = result_prob2.select("opportunity_id","sales_agent","product","final_price","actual_price")
result_prob2.show()
print (result_prob2.count())

+--------------+-----------------+----------+------------+-----------+------------+
|opportunity_id|      sales_agent|deal_stage|     product|final_price|actual_price|
+--------------+-----------------+----------+------------+-----------+------------+
|      67HY0MW7|    Donn Cantrell|       Won|    GTXBasic|      500.0|       641.0|
|      R22O68FF|  Niesha Huffines|       Won|    GTXBasic|      524.0|       641.0|
|      J78AK31N|    Kami Bicknell|       Won|      MGRPFU|     4794.0|      3959.0|
|      8I9PRPGN|Versie Hillebrand|       Won|      MGRPFS|       67.0|        64.0|
|      4VHUTHOJ|    Kami Bicknell|       Won|GTXPlusBasic|     1480.0|      1279.0|
|      TMJ0OJ0B|  Kary Hendrixson|       Won|    GTXBasic|      635.0|       641.0|
|      MD4PBMNN|    Anna Snelling|       Won|      MGRPFU|     3842.0|      3959.0|
|      1XPVT5AY|  Kary Hendrixson|       Won|  GTXPlusPro|     5055.0|      6395.0|
|      IH2QISS9|  Kary Hendrixson|       Won|      GTXPro|     4889.0|      

PROBLEM 3: Display the 'Opportunity ID' and 'Days Taken to Close', for opportunities those got closed within a month

In [7]:
#solution code here
#Filter on deal_stage and parsing the date column to date type and using datediff
result_prob3 = df_sales_pipeline.filter(col("deal_stage")=="Won")\
                                .withColumn("DaysTakenToClose", 
                                            datediff(to_date("close_date","yyyy-MM-dd"),
                                                     to_date("created_on","yyyy-MM-dd"))).select("opportunity_id","DaysTakenToClose")
result_prob3.show(30)
print (result_prob3.count())

#hardcoding the range of month as 30 for now
result_prob3 = result_prob3.filter(col("DaysTakenToClose") <= 30)
result_prob3.show(30)
print (result_prob3.count())

+--------------+----------------+
|opportunity_id|DaysTakenToClose|
+--------------+----------------+
|      67HY0MW7|              12|
|      R22O68FF|              98|
|      J78AK31N|              81|
|      8I9PRPGN|              41|
|      4VHUTHOJ|               1|
|      TMJ0OJ0B|              10|
|      MD4PBMNN|              95|
|      1XPVT5AY|              57|
|      IH2QISS9|              94|
|      7JJ73XCX|              61|
|      T8QRTV6F|              10|
|      C7NFUAR6|              65|
|      NXRZZBVS|              52|
|      H7ZQUWDJ|               9|
|      YV47EGIC|              65|
|      SC4LUMPZ|              53|
|      ZES3NR0F|              89|
|      KJ1JOOQ0|              13|
|      ZHV68QKO|              89|
|      4RE1ST7V|              22|
|      8S4H8GNZ|              65|
|      P007M3B8|             109|
|      KI49INQW|              50|
|      2NAKBIH8|              87|
|      QRYFOK47|              36|
|      3X1QOEBM|              68|
|      TGYPRG6

PROBLEM 4: Display product(s) got maximum leads (by count) generated from paid source

In [8]:
#solution code here


PROBLEM 5: Display 'Sales Agent' and 'Opportunity Count', for those sales agents who lost atleast two opportunties

In [9]:
#solution code here
result_prob5 = df_sales_pipeline.filter(col("deal_stage")=="Lost")\
                                .select("sales_agent","opportunity_id")\
                                .groupBy("sales_agent").agg(count("opportunity_id").alias("numOfLostOpportunities"))
result_prob5.show(20)
print (result_prob5.count())

result_prob5 = result_prob5.filter(col("numOfLostOpportunities") >= 2)

result_prob5.show(20)
print (result_prob5.count())

#validation test
print ("validation test: ", result_prob5.agg(sum("numOfLostOpportunities")).collect()[0][0] == df_sales_pipeline.filter(col("deal_stage")=="Lost").count())

+------------------+----------------------+
|       sales_agent|numOfLostOpportunities|
+------------------+----------------------+
|   Darcel Schlecht|                   337|
|     Kami Bicknell|                   134|
|    Vicki Laflamme|                   162|
|      Elease Gluck|                    62|
|Jonathan Berthelot|                   185|
|   Daniell Hammack|                    80|
|     Anna Snelling|                   140|
|      Cassey Cress|                   137|
|     Garret Kinder|                    63|
|    Markita Hansen|                   115|
|      Reed Clapper|                    87|
|Rosie Papadopoulos|                    56|
|   Maureen Marcano|                   119|
|  Violet Mclelland|                   111|
|  Gladys Colclough|                   149|
|         Boris Faz|                    63|
|    Wilburn Farren|                    44|
| Versie Hillebrand|                   118|
| Marty Freudenburg|                   120|
|    Cecily Lampkin|            

PROBLEM 6: Display in ascending order of revenue, 'Account' and 'Revenue' for telecom accounts 

In [10]:
#solution code here
df_accounts = df_accounts.withColumn("revenue",df_accounts["revenue"].cast(DoubleType()))
result_prob6 = df_accounts.orderBy("revenue",ascending=True).select("account","revenue")
result_prob6.show(30)
print (result_prob6.count())

+--------------------+-------+
|             account|revenue|
+--------------------+-------+
|          Stanredtax|  14.79|
|          Fasehatice|   19.2|
|            Kan-code|  22.63|
|           Treequote|   73.1|
|           Konmatfix|  82.96|
|Olivia Pope & Ass...|  97.94|
|         Donquadtech| 110.88|
|           Warephase| 130.62|
|        Soylent Corp| 136.89|
|         Iselectrics| 138.63|
|              Yearin| 144.68|
|           Ganjaflex|  161.8|
|     Sterling Cooper| 204.47|
|            Rangreen| 211.12|
|            Xx-zobam| 221.86|
|              Hatfan| 223.54|
|            Betatech| 239.22|
|           Duff Beer| 244.32|
|         Good Burger| 247.91|
|            Blackzim| 256.32|
|             Nam-zim| 369.58|
|         Krusty Krab| 372.21|
|               Hooli| 374.84|
|             Finhigh| 398.18|
|            Hottechi|  431.1|
|          Bubba Gump| 459.79|
|          Green-Plus| 562.28|
|              Sonron| 588.46|
|             Codehow| 638.03|
|       

PROBLEM 7: Display by revenue generated, bottom five 'Industries' and 'Revenue'

In [11]:
#solution code here
result_prob7 = df_accounts.orderBy("revenue",ascending=False).select("account","revenue")
result_prob7.show(20)
print (result_prob7.count())
#generating id incrementally for accounts sorted by revenue in decreasing order
result_prob7 = result_prob7.withColumn("index", monotonically_increasing_id())
#sorting by index in descending order so that we get accounts with least revenue (bottom 5)
result_prob7.orderBy(desc("index")).drop("index").show(5)

+-----------------+-------+
|          account|revenue|
+-----------------+-------+
|   Dalttechnology| 6085.6|
|            Silis|5339.57|
|            Newex|5093.78|
| Wonka Industries|4962.27|
|         Faxquote|4939.54|
|       Sunnamplex|4592.96|
|            Isdom|4514.68|
|          Golddex|4340.32|
|          Conecom|4242.85|
| Stark Industries|4221.65|
|         Scotfind|3911.27|
|         Ron-tech|3805.02|
|         Gogozoom| 3577.1|
|       Bioholding| 3321.9|
|         Zumgoity| 3264.4|
|Wayne Enterprises|3193.45|
|         Dontechi|2990.17|
|       Gekko & Co|2934.79|
|       Zathunicon|2913.82|
|         Labdrill|2913.26|
+-----------------+-------+
only showing top 20 rows

97
+----------+-------+
|   account|revenue|
+----------+-------+
|Stanredtax|  14.79|
|Fasehatice|   19.2|
|  Kan-code|  22.63|
| Treequote|   73.1|
| Konmatfix|  82.96|
+----------+-------+
only showing top 5 rows



PROBLEM 8: Display 'Month of Year' vs 'Sales', for GTXBasic. NOTE: "Month of Year" means month year (eg. Jan) and "Month" means month (eg. Jan 2020, Jan 2021 etc)

In [12]:
#solution code here
#formatting created_on (sales initiation date)
result_prob8 = df_sales_pipeline.filter(col("product")=='GTXBasic')\
                                .select("opportunity_id","deal_stage","product","created_on","close_date")\
                                .withColumn("Month Of year",date_format("created_on","LLL y"))
result_prob8.show(30)
print (result_prob8.count())

+--------------+-----------+--------+----------+----------+-------------+
|opportunity_id| deal_stage| product|created_on|close_date|Month Of year|
+--------------+-----------+--------+----------+----------+-------------+
|      67HY0MW7|        Won|GTXBasic|2017-04-24|2017-05-06|     Apr 2017|
|      BRL1KVVH|       Lost|GTXBasic|2017-05-19|2017-08-03|     May 2017|
|      R22O68FF|        Won|GTXBasic|2017-03-21|2017-06-27|     Mar 2017|
|      TMJ0OJ0B|        Won|GTXBasic|2017-06-30|2017-07-10|     Jun 2017|
|      B22V5Z3B|       Lost|GTXBasic|2017-07-25|2017-08-07|     Jul 2017|
|      H7ZQUWDJ|        Won|GTXBasic|2017-12-21|2017-12-30|     Dec 2017|
|      HC77MZU1|       Lost|GTXBasic|2017-07-25|2017-09-07|     Jul 2017|
|      27QUC5M7|In_Progress|GTXBasic|2017-11-01|      null|     Nov 2017|
|      N3DXP5OP|In_Progress|GTXBasic|2017-02-05|      null|     Feb 2017|
|      ZLLF7UHU|       Lost|GTXBasic|2017-09-20|2017-12-08|     Sep 2017|
|      QRYFOK47|        Won|GTXBasic|2

PROBLEM 9: Which sales agent(s) never lost a deal. Display as a dictionary {sales agent:sales}

In [13]:
#solution code here
df_sales_notlost = df_sales_pipeline.filter(col("deal_stage") != "Lost").select("opportunity_id","sales_agent","close_value")
result_prob9 = defaultdict(list)
for row in df_sales_notlost.toJSON().collect():
    row_json = json.loads(row)
    if "close_value" in row_json:
        result_prob9[row_json['sales_agent']].append({"opportunity_id":row_json['opportunity_id'],
                                                      "close_value":row_json["close_value"]})
    else:
        result_prob9[row_json['sales_agent']].append({"opportunity_id":row_json['opportunity_id'],
                                                      "close_value":None})
print (result_prob9.keys())
print (len(result_prob9.keys()))

dict_keys(['Donn Cantrell', 'James Ascencio', 'Niesha Huffines', 'Kami Bicknell', 'Versie Hillebrand', 'Kary Hendrixson', 'Anna Snelling', 'Vicki Laflamme', 'Darcel Schlecht', 'Marty Freudenburg', 'Lajuana Vencill', 'Reed Clapper', 'Elease Gluck', 'Rosie Papadopoulos', 'Jonathan Berthelot', 'Zane Levy', 'Rosalina Dieter', 'Cecily Lampkin', 'Gladys Colclough', 'Daniell Hammack', 'Moses Frase', 'Wilburn Farren', 'Violet Mclelland', 'Markita Hansen', 'Corliss Cosme', 'Cassey Cress', 'Boris Faz', 'Maureen Marcano', 'Hayden Neloms', 'Garret Kinder'])
30


PROBLEM 10: Display 'Sales Agents', 'Product', and 'Sales', for those sales agents who closed more than one deal on same day

In [14]:
#solution code here
result_prob10 = df_sales_pipeline.filter(col("deal_stage")=="Won")\
                                 .withColumn("DaysTakenToClose", 
                                            datediff(to_date("close_date","yyyy-MM-dd"),
                                                     to_date("created_on","yyyy-MM-dd")))\
                                 .select("opportunity_id","sales_agent","product","DaysTakenToClose")

result_prob10.show(20)
print (result_prob10.count())
result_prob10 = result_prob10.filter(col("DaysTakenToClose") == 1)\
                             .select("sales_agent","product","opportunity_id")\
                             .groupBy("sales_agent","product").agg(count("opportunity_id").alias("completed_opportunities"))\
                             .filter(col("completed_opportunities") > 1)

result_prob10.show(20)
print (result_prob10.count())

+--------------+-----------------+------------+----------------+
|opportunity_id|      sales_agent|     product|DaysTakenToClose|
+--------------+-----------------+------------+----------------+
|      67HY0MW7|    Donn Cantrell|    GTXBasic|              12|
|      R22O68FF|  Niesha Huffines|    GTXBasic|              98|
|      J78AK31N|    Kami Bicknell|      MGRPFU|              81|
|      8I9PRPGN|Versie Hillebrand|      MGRPFS|              41|
|      4VHUTHOJ|    Kami Bicknell|GTXPlusBasic|               1|
|      TMJ0OJ0B|  Kary Hendrixson|    GTXBasic|              10|
|      MD4PBMNN|    Anna Snelling|      MGRPFU|              95|
|      1XPVT5AY|  Kary Hendrixson|  GTXPlusPro|              57|
|      IH2QISS9|  Kary Hendrixson|      GTXPro|              94|
|      7JJ73XCX|   James Ascencio|GTXPlusBasic|              61|
|      T8QRTV6F|   James Ascencio|  GTXPlusPro|              10|
|      C7NFUAR6|Marty Freudenburg|GTXPlusBasic|              65|
|      NXRZZBVS|  Lajuana

Refer & Use Orchestra.json to answer problem 11-13 below

PROBLEM 11: Display the instrument played by Lehmann Caroline

In [15]:
#solution code here
#Edited the orchestra.json file to retrieve the proper JSON structure to accommodate Array of JSON objects
df_orchestra = spark.read.format("json").option("multiline","true").load("data/Orchestra.json")
df_orchestra.show(20)
print (df_orchestra.count())

+--------------------+--------------------+--------------------+---------+-------+--------------------+
|            concerts|                  id|           orchestra|programID| season|               works|
+--------------------+--------------------+--------------------+---------+-------+--------------------+
|[{1842-12-07T05:0...|00646b9f-fec7-4ff...|New York Philharm...|     3853|1842-43|[{52446*, Beethov...|
|[{1843-02-18T05:0...|1118e84e-eb59-46c...|New York Philharm...|     5178|1842-43|[{52437*, Beethov...|
|[{1843-04-07T05:0...|08536612-27c3-437...|Musicians from th...|    10785|1842-43|[{52364*1, Beetho...|
|[{1843-04-22T05:0...|81a3b8de-1737-4c9...|New York Philharm...|     5887|1842-43|[{52434*, Beethov...|
|[{1843-11-18T05:0...|09581bb7-8855-496...|New York Philharm...|      305|1843-44|[{52453*, Beethov...|
|[{1844-01-13T05:0...|0848266c-8eee-48a...|New York Philharm...|     3368|1843-44|[{51668*, Mozart,...|
|[{1844-03-16T05:0...|8025e763-9c12-415...|New York Philharm...|

In [16]:
df_orchestra.dtypes

[('concerts',
  'array<struct<Date:string,Location:string,Time:string,Venue:string,eventType:string>>'),
 ('id', 'string'),
 ('orchestra', 'string'),
 ('programID', 'string'),
 ('season', 'string'),
 ('works',
  'array<struct<ID:string,composerName:string,conductorName:string,interval:string,movement:string,soloists:array<struct<soloistInstrument:string,soloistName:string,soloistRoles:string>>,workTitle:string>>')]

In [17]:
def getRequiredInstrument(ws,soloist_name):
    exists = None
    for work in ws:
        if len(work['soloists']) > 0:
            for soloist_element in work['soloists']:
                if soloist_element["soloistName"] == soloist_name:
                    exists = 1
                    break
            if exists == 1:
                break
        else:
            continue
    if exists:
        return soloist_element["soloistInstrument"]
    else:
        return ""

instrumentUDF = udf(lambda colname,sname: getRequiredInstrument(colname,sname))

In [18]:
test = df_orchestra.withColumn("return_instrument",instrumentUDF("works",lit("Lehmann, Caroline")))
test.select("id","orchestra","works","programID","return_instrument").filter(col("return_instrument")!="").show(40)

+--------------------+--------------------+--------------------+---------+-----------------+
|                  id|           orchestra|               works|programID|return_instrument|
+--------------------+--------------------+--------------------+---------+-----------------+
|21b49b4a-1805-41b...|New York Philharm...|[{52437*, Beethov...|     5792|          Soprano|
|89c1db0d-8c41-476...|New York Philharm...|[{52577*, Mendels...|     2149|          Soprano|
|4e81347f-898d-48b...|New York Philharm...|[{52374*1, Cherub...|     6613|          Soprano|
+--------------------+--------------------+--------------------+---------+-----------------+



PROBLEM 12: Display all vocalists

In [19]:
#solution code here
df_orchestra.select("works.composerName").distinct().take(10)

[Row(composerName=['Beethoven,  Ludwig  van', 'Weber,  Carl  Maria Von', 'Hummel,  Johann', 'Pacini,  Giovanni', 'Romberg,  Bernhard', 'Onslow,  George', 'Onslow,  George', 'Rossini,  Gioachino', 'Thalberg,  Sigismond', 'Mozart,  Wolfgang  Amadeus', 'Herz,  Henri', 'Lindpaintner,  Peter  Von']),
 Row(composerName=['Beethoven,  Ludwig  van', 'Vieuxtemps,  Henri', 'Mendelssohn,  Felix', 'Donizetti,  Gaetano', 'Weber,  Carl  Maria Von']),
 Row(composerName=['Beethoven,  Ludwig  van', None, 'Loder,  George, Jr.', 'Mendelssohn,  Felix', 'Lindpaintner,  Peter  Von', 'Beriot,  Charles-Auguste  de', 'Reissiger,  Karl  Gottlieb']),
 Row(composerName=['Schumann,  Robert', 'Haydn,  Franz  Joseph', 'Ernst,  Heinrich  Wilhelm', None, 'Rietz,  Julius', 'Weber,  Carl  Maria Von', 'Mollenhauer,  Friedrich', None, 'Beethoven,  Ludwig  van']),
 Row(composerName=['Strauss,  Richard', 'Beethoven,  Ludwig  van', None, 'Volkmann,  Friedrich  Robert', 'Schumann,  Robert']),
 Row(composerName=['Wagner,  Richa

PROBLEM 13: Display orchestra played under program id 2561

In [20]:
#solution code here
df_orchestra.select("programID","orchestra").filter(col("programID")=="2561").show()

+---------+--------------------+
|programID|           orchestra|
+---------+--------------------+
|     2561|New York Philharm...|
+---------+--------------------+



Refer & Use Orchestra.xml to answer problem 14-15 below

PROBLEM 14: Display locations used for event at time 8:15 PM

In [21]:
#solution code here
def get_locations(concerts,time):
    locations = []
    if len(concerts) > 0:
        for concert in concerts:
            if concert['Time'] == '8:15PM':
                locations.append(concert['Location'])
    if len(locations) > 0:
        return locations
    else:
        return ""

locationsUDF = udf(lambda colname,sname: get_locations(colname,sname))

In [22]:
test = df_orchestra.withColumn("return_location",locationsUDF("concerts",lit("8:15PM")))
test.select("id","orchestra","concerts","programID","return_location").filter(col("return_location")!="").show(40)

+--------------------+--------------------+--------------------+---------+---------------+
|                  id|           orchestra|            concerts|programID|return_location|
+--------------------+--------------------+--------------------+---------+---------------+
|1124c816-f97c-4cf...|   New York Symphony|[{1888-12-28T05:0...|    11610|[Manhattan, NY]|
|7e1d590a-9750-4e0...|   New York Symphony|[{1889-12-13T05:0...|     7921|[Manhattan, NY]|
|ebe38ed5-af8a-42c...|   New York Symphony|[{1890-01-31T05:0...|     8072|[Manhattan, NY]|
|fa51a57c-f27c-430...|   New York Symphony|[{1891-11-17T05:0...|    14304|[Manhattan, NY]|
|20df0162-d9ad-407...|   New York Symphony|[{1891-11-19T05:0...|    14305|[Manhattan, NY]|
|c5f00d41-3ae7-499...|   New York Symphony|[{1892-01-15T05:0...|     8087|[Manhattan, NY]|
|de6b31a7-d117-47e...|   New York Symphony|[{1892-02-05T05:0...|     8088|[Manhattan, NY]|
|637c6a11-0e03-48c...|   New York Symphony|[{1892-03-04T05:0...|     8089|[Manhattan, NY]|

In [23]:
test.filter(col("return_location")!="").select("return_location").distinct().show(40)

+--------------------+
|     return_location|
+--------------------+
|   [Springfield, MA]|
|        [Newark, NJ]|
|     [Manhattan, NY]|
|    [Providence, RI]|
|  [Indianapolis, IN]|
|     [New Haven, CT]|
|      [Brooklyn, NY]|
|  [Philadelphia, PA]|
|      [Hartford, CT]|
|[Manhattan, NY, B...|
|[Manhattan, NY, M...|
+--------------------+



PROBLEM 15: Display total number of programs

In [24]:
#solution code here
df_orchestra.count()

1033

********************************************* Test ends here **************************************************